In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import mne
from scipy.signal import welch
import warnings

# ========== CONFIG ==========
DATA_ROOT = Path(r"C:\Users\alhrb\Downloads\ecg\SeizeIT2")
FS_TARGET = 256
WIN_SEC = 2
PRE_ICTAL_SEC = 15 * 60  # 15 minutes
NOTCH_FREQ = 50
BP_LOW, BP_HIGH = 0.5, 45
OUTPUT_CSV = "ECG.csv"
STATS_CSV = "ECG_Run_Statistics.csv"
SEIZURE_CSV = Path(r"C:\Users\alhrb\Downloads\ecg\seizure_runs_details.csv")

warnings.filterwarnings("ignore")
mne.set_log_level("ERROR")

# ========== LOAD SEIZURE METADATA ==========
seizure_df = pd.read_csv(SEIZURE_CSV)

# ========== FUNCTIONS ==========

def load_ecg(ecg_path):
    raw = mne.io.read_raw_edf(ecg_path, preload=True)
    raw.resample(FS_TARGET)
    return raw

def apply_notch(raw, freq=50):
    return raw.copy().notch_filter(freqs=[freq])

def bandpass(raw, low=0.5, high=45):
    return raw.copy().filter(l_freq=low, h_freq=high)

def extract_features(window):
    if np.std(window) == 0:
        return [np.nan] * 11

    mean_val = np.mean(window)
    std_val = np.std(window)
    max_val = np.max(window)
    min_val = np.min(window)
    ptp = max_val - min_val
    energy = np.sum(window ** 2)

    freqs, psd = welch(window, fs=FS_TARGET, nperseg=len(window))

    def band_power(low, high):
        mask = (freqs >= low) & (freqs <= high)
        return np.trapz(psd[mask], freqs[mask]) if np.any(mask) else 0.0

    delta = band_power(0.5, 4)
    theta = band_power(4, 8)
    alpha = band_power(8, 13)
    beta = band_power(13, 45)

    dominant_freq = np.nan if np.all(np.isnan(psd)) or np.all(psd == 0) else freqs[np.argmax(psd)]

    return [mean_val, std_val, max_val, min_val, ptp, energy,
            delta, theta, alpha, beta, dominant_freq]

def label_window(win_t_start, seizures):
    for onset, duration in seizures:
        if onset <= win_t_start <= onset + duration:
            return 2
        if (onset - PRE_ICTAL_SEC) <= win_t_start < onset:
            return 1
    return 0

def process_run(sub, ses, run, ecg_path, seizures):
    raw = load_ecg(ecg_path)
    raw = apply_notch(raw, NOTCH_FREQ)
    raw = bandpass(raw, BP_LOW, BP_HIGH)

    sig = raw.get_data()[0]
    t = raw.times

    win_samples = int(WIN_SEC * FS_TARGET)
    total_samples = len(sig)
    n_windows = (total_samples - win_samples) // win_samples

    rows = []
    label_counts = {0: 0, 1: 0, 2: 0}

    print("\n============================")
    print(f"➡ Run — Processing: {sub} | {ses} | run-{run:02d}")
    print(f"Fs={FS_TARGET}, Samples={total_samples}")

    for i, start in enumerate(range(0, total_samples - win_samples+1, win_samples)):
        end = start + win_samples
        window = sig[start:end]
        win_time = t[start]

        label = label_window(win_time, seizures)
        label_counts[label] += 1
        feats = extract_features(window)

        rows.append([sub, ses, run, i, win_time, label] + feats)

    total_ictal_duration = sum([d for (_, d) in seizures])

    print("\n[Window Counts]")
    print(f"🟦 Normal:    {label_counts[0]} windows")
    print(f"🟨 Pre-ictal: {label_counts[1]} windows")
    print(f"🟥 Ictal:     {label_counts[2]} windows")
    print(f"\n➡ Finished Run — Total windows: {sum(label_counts.values())}")

    stats = {
        "subject": sub,
        "session": ses,
        "run": run,
        "n_normal": label_counts[0],
        "n_pre_ictal": label_counts[1],
        "n_ictal": label_counts[2],
        "ictal_duration_sec": total_ictal_duration
    }

    return rows, stats

# ========== MAIN LOOP ==========
all_rows = []
all_stats = []

print("\n✨ Starting ECG Feature Extraction with Statistics...\n")

# ✅ Group by (subject, session, run) to avoid duplicates
grouped = seizure_df.groupby(["subject", "session", "run"])

for (sub, ses, run), group in grouped:
    seizures = list(zip(group["onset"], group["duration"]))

    ecg_dir = DATA_ROOT / sub / ses / "ecg"
    ecg_filename = f"{sub}_{ses}_task-szMonitoring_run-{int(run):02d}_ecg.edf"
    ecg_path = ecg_dir / ecg_filename

    if not ecg_path.exists():
        print(f"⚠️ ECG missing: {ecg_filename}")
        continue

    run_rows, run_stats = process_run(sub, ses, run, ecg_path, seizures)
    all_rows.extend(run_rows)
    all_stats.append(run_stats)

# ========== SAVE FEATURES ==========
columns = [
    "subject", "session", "run", "window_idx", "start_time_sec", "label",
    "ecg_mean", "ecg_std", "ecg_max", "ecg_min", "ecg_ptp", "ecg_energy",
    "ecg_delta_power", "ecg_theta_power", "ecg_alpha_power",
    "ecg_beta_power", "ecg_dominant_freq"
]

df = pd.DataFrame(all_rows, columns=columns)
df.to_csv(OUTPUT_CSV, index=False)

# ========== SAVE STATS ==========
stats_df = pd.DataFrame(all_stats)
stats_df.to_csv(STATS_CSV, index=False)

print("\n🌟 ECG Feature Extraction Completed!")
print("📁 Features File:", OUTPUT_CSV)
print("📊 Stats File   :", STATS_CSV)
print("\n📊 Sample Statistics Preview:")
print(stats_df.head())



✨ Starting ECG Feature Extraction with Statistics...


➡ Run — Processing: sub-001 | ses-01 | run-03
Fs=256, Samples=16634624

[Window Counts]
🟦 Normal:    32003 windows
🟨 Pre-ictal: 450 windows
🟥 Ictal:     36 windows

➡ Finished Run — Total windows: 32489

➡ Run — Processing: sub-001 | ses-01 | run-05
Fs=256, Samples=16002816

[Window Counts]
🟦 Normal:    30764 windows
🟨 Pre-ictal: 450 windows
🟥 Ictal:     41 windows

➡ Finished Run — Total windows: 31255

➡ Run — Processing: sub-001 | ses-01 | run-07
Fs=256, Samples=7556608

[Window Counts]
🟦 Normal:    14237 windows
🟨 Pre-ictal: 450 windows
🟥 Ictal:     72 windows

➡ Finished Run — Total windows: 14759

➡ Run — Processing: sub-001 | ses-01 | run-08
Fs=256, Samples=13178368

[Window Counts]
🟦 Normal:    25276 windows
🟨 Pre-ictal: 450 windows
🟥 Ictal:     13 windows

➡ Finished Run — Total windows: 25739

➡ Run — Processing: sub-002 | ses-01 | run-01
Fs=256, Samples=2913280

[Window Counts]
🟦 Normal:    4688 windows
🟨 Pre-ictal: 900